In [1]:
# SEGMENTACTION --- takes in input the complete stream of an action and 
#                   separates it into instances. The user can use this 
#                   function to segment Scene files.
#
#              Input: 
#                   - filename: path of the mat structure of the action
#              
#              This function will
#                   - take the 3d streams of all markers of the action and 
#                       segment it into instances following the cut indices 
#                       stored in the 'index' field
#                   - save a structure array containing all the instances 
#                       in temporal order, for every marker
#              
#              Output:
#                   - files named *action*_instance_number.csv will be saved.
#                   
# Example of use:
# action = '../CookingDataset-master/data/mocap/'
# mode="training" # or 'test'
# action="carrot" 
# output_folder="./data/segmentAction/"
# segmentAction(folder,mode,action,output_folder)

import os
from os import path
import sys
import csv
import pandas as pd
from scipy.io import loadmat 

def segmentAction(folder, mode, action,output_folder):
    if not os.path.exists(output_folder): #to save files
        os.makedirs(output_folder)
        
    if mode == 'training':
        file =  path.join(folder,mode, (action+'_tr.mat'))
        name=action+'_tr_'
        
    elif mode == 'test':
        file =  path.join(folder,mode, (action+'_test.mat'))
        name=action+'_test_'
        
    single_action = loadmat(file)  # load mat-file
    
    all_column_names=['IND_X','IND_Y','IND_Z','PALM_X','PALM_Y','PALM_Z', 'LIT_X', 'LIT_Y','LIT_Z','WRIST_X','WRIST_Y','WRIST_Z', 'ELBOW_X','ELBOW_Y','ELBOW_Z', 'SHOULDER_X','SHOULDER_Y','SHOULDER_Z']
    all_marker = ['IND','PALM', 'LIT', 'WRIST', 'ELBOW', 'SHOULDER']
    index=single_action['index']
    index_len= index.shape[1]
#     print(len(index))
    df = pd.DataFrame(columns = all_column_names)
    for a in all_marker:
            arr = single_action[a]
            x=a+'_X'
            y=a+'_Y'
            z=a+'_Z'
            df[x]=arr[:,0]
            df[y]=arr[:,1]
            df[z]=arr[:,2]
    
    for i in range( 0,index_len-1):
        start= index[0][i]
        end=index[0][i+1]
        new_df=df.iloc[start-1:end]
        f=path.join(output_folder,(name+str(i+1).zfill(3)+'.csv'))
        new_df.to_csv(f,index = False)
        

    

In [2]:
# Function Calling
# all_action=['carrot','cutting','dish','eating','eggs','lemon','mezzaluna','mixing','openbottle','pan','pestling','pouringmult','pouringsingle','reaching','rolling','salad','salting','spreading','table','transporting']

folder = "../CookingDataset-master/data/mocap/"  # cooking dataset folder
mode="training" # or 'test'
action_list=["carrot",'dish'] # list of actions to segment
output_folder="./data/segmentAction/"

for action in action_list:
    segmentAction(folder,mode,action,output_folder)